In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DateType, DoubleType

catalog_name = "nyc_yellow_taxi"

In [0]:
silver_df = spark.table(f"{catalog_name}.silver.trips")
display(silver_df.limit(10))

vendor_id,pickup_ts,dropoff_ts,trip_duration_minutes,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,cbd_congestion_fee,ingestion_time,_source_file
2,2025-01-01T00:39:43.000,2025-01-01T00:48:22.000,8,2,1.88,1,N,143,151,1,10.7,1.0,0.5,3.14,0.0,1.0,18.84,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet
2,2025-01-01T00:21:46.000,2025-01-01T00:32:08.000,10,1,1.43,1,N,263,238,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet
2,2025-01-01T00:45:41.000,2025-01-01T01:05:40.000,19,1,2.96,1,N,114,141,1,19.8,1.0,0.5,2.48,0.0,1.0,27.28,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet
2,2025-01-01T00:07:17.000,2025-01-01T00:14:17.000,7,1,1.31,1,N,158,107,1,9.3,1.0,0.5,1.86,0.0,1.0,16.16,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet
2,2025-01-01T01:25:10.000,2025-01-01T01:42:40.000,17,2,6.32,1,N,231,237,1,27.5,1.0,0.5,4.88,0.0,1.0,37.38,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet
2,2025-01-01T01:56:09.000,2025-01-01T02:01:48.000,5,1,0.93,1,N,113,158,1,7.2,1.0,0.5,1.5,0.0,1.0,13.7,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet
2,2025-01-01T01:00:45.000,2025-01-01T01:10:11.000,9,1,1.74,1,N,79,164,1,10.7,1.0,0.5,1.0,0.0,1.0,16.7,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet
1,2025-01-01T01:08:19.000,2025-01-01T01:28:00.000,19,1,3.1,1,N,236,264,1,20.5,3.5,0.5,6.35,0.0,1.0,31.85,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet
2,2025-01-01T01:51:15.000,2025-01-01T02:04:12.000,12,1,3.58,1,N,87,189,1,17.7,1.0,0.5,4.54,0.0,1.0,27.24,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet
2,2025-01-01T01:29:34.000,2025-01-01T01:48:03.000,18,1,3.09,1,N,164,236,1,20.5,1.0,0.5,5.1,0.0,1.0,30.6,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet


In [0]:
df = spark.sql(f"""
SELECT *,
    CASE WHEN vendor_id = 1 THEN 'Creative Mobile Technologies, LLC'
        WHEN vendor_id = 2 THEN 'Curb Mobility'
        WHEN vendor_id = 6 THEN 'Myle Technologies Inc'
        WHEN vendor_id = 7 THEN 'Helix'
        ELSE 'Unknown'
        END AS vendor_name,
    CASE WHEN payment_type_id = 0 THEN 'Flex Fare trip'
        WHEN payment_type_id = 1 THEN 'Credit card'
        WHEN payment_type_id = 2 THEN 'Cash'
        WHEN payment_type_id = 3 THEN 'No charge'
        WHEN payment_type_id = 4 THEN 'Dispute'
        WHEN payment_type_id = 5 THEN 'Unknown'
        WHEN payment_type_id = 6 THEN 'Voided trip'
        ELSE 'Unknown'
        END AS payment_description,
    CASE WHEN rate_code_id = 1 THEN 'Standard rate'
        WHEN rate_code_id = 2 THEN 'JFK'
        WHEN rate_code_id = 3 THEN 'Newark'
        WHEN rate_code_id = 4 THEN 'Nassau or Westchester'
        WHEN rate_code_id = 5 THEN 'Negotiated fare'
        WHEN rate_code_id = 6 THEN 'Group ride'
        ELSE 'Unknown'
        END AS rate_code,
    dayofweek(pickup_ts) AS pu_dow_number,
    date_format(pickup_ts, 'EEEE') AS pu_weekday,
    CASE WHEN HOUR(pickup_ts) BETWEEN 5 and 11 THEN "Morning"
        WHEN HOUR(pickup_ts) BETWEEN 12 and 16 THEN "Afternoon"
        WHEN HOUR(pickup_ts) BETWEEN 17 and 21 THEN "Evening"
        ELSE "Night" END AS pu_time_of_day
FROM
    {catalog_name}.silver.trips
""")

display(df.limit(10))


vendor_id,pickup_ts,dropoff_ts,trip_duration_minutes,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,cbd_congestion_fee,ingestion_time,_source_file,vendor_name,payment_description,rate_code,pu_dow_number,pu_weekday,pu_time_of_day
2,2025-01-01T00:39:43.000,2025-01-01T00:48:22.000,8,2,1.88,1,N,143,151,1,10.7,1.0,0.5,3.14,0.0,1.0,18.84,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,Curb Mobility,Credit card,Standard rate,4,Wednesday,Night
2,2025-01-01T00:21:46.000,2025-01-01T00:32:08.000,10,1,1.43,1,N,263,238,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,Curb Mobility,Credit card,Standard rate,4,Wednesday,Night
2,2025-01-01T00:45:41.000,2025-01-01T01:05:40.000,19,1,2.96,1,N,114,141,1,19.8,1.0,0.5,2.48,0.0,1.0,27.28,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,Curb Mobility,Credit card,Standard rate,4,Wednesday,Night
2,2025-01-01T00:07:17.000,2025-01-01T00:14:17.000,7,1,1.31,1,N,158,107,1,9.3,1.0,0.5,1.86,0.0,1.0,16.16,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,Curb Mobility,Credit card,Standard rate,4,Wednesday,Night
2,2025-01-01T01:25:10.000,2025-01-01T01:42:40.000,17,2,6.32,1,N,231,237,1,27.5,1.0,0.5,4.88,0.0,1.0,37.38,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,Curb Mobility,Credit card,Standard rate,4,Wednesday,Night
2,2025-01-01T01:56:09.000,2025-01-01T02:01:48.000,5,1,0.93,1,N,113,158,1,7.2,1.0,0.5,1.5,0.0,1.0,13.7,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,Curb Mobility,Credit card,Standard rate,4,Wednesday,Night
2,2025-01-01T01:00:45.000,2025-01-01T01:10:11.000,9,1,1.74,1,N,79,164,1,10.7,1.0,0.5,1.0,0.0,1.0,16.7,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,Curb Mobility,Credit card,Standard rate,4,Wednesday,Night
1,2025-01-01T01:08:19.000,2025-01-01T01:28:00.000,19,1,3.1,1,N,236,264,1,20.5,3.5,0.5,6.35,0.0,1.0,31.85,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,"Creative Mobile Technologies, LLC",Credit card,Standard rate,4,Wednesday,Night
2,2025-01-01T01:51:15.000,2025-01-01T02:04:12.000,12,1,3.58,1,N,87,189,1,17.7,1.0,0.5,4.54,0.0,1.0,27.24,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,Curb Mobility,Credit card,Standard rate,4,Wednesday,Night
2,2025-01-01T01:29:34.000,2025-01-01T01:48:03.000,18,1,3.09,1,N,164,236,1,20.5,1.0,0.5,5.1,0.0,1.0,30.6,2.5,0.0,0.0,2026-01-19T14:46:14.334Z,dbfs:/Volumes/nyc_yellow_taxi/source_data/raw/2025_data/yellow_tripdata_2025-01.parquet,Curb Mobility,Credit card,Standard rate,4,Wednesday,Night


In [0]:
df.createOrReplaceTempView("gold_df")

df_summary = spark.sql(f"""
SELECT 
    pu_weekday,
    pu_dow_number,
    pu_time_of_day,
    COUNT(*) AS total_trips,
    ROUND(SUM(total_amount), 2) AS daily_revenue,
    ROUND(AVG(CASE WHEN fare_amount > 0 THEN (tip_amount / fare_amount) * 100 ELSE 0 END), 2) AS tip_pct
FROM gold_df
GROUP BY pu_weekday, pu_dow_number, pu_time_of_day
ORDER BY pu_dow_number;
                       """)

display(df_summary)
df_summary.write.mode("overwrite").saveAsTable(f"{catalog_name}.gold.trip_summary")
df.write.mode("overwrite").saveAsTable(f"{catalog_name}.gold.trips")

pu_weekday,pu_dow_number,pu_time_of_day,total_trips,daily_revenue,tip_pct
Sunday,1,Afternoon,106643,2924052.95,21.48
Sunday,1,Morning,56747,1555768.84,20.86
Sunday,1,Evening,82042,2315563.72,22.16
Sunday,1,Night,67747,1846307.28,22.5
Monday,2,Evening,97042,2784979.43,24.54
Monday,2,Night,28453,1028044.97,21.15
Monday,2,Morning,80150,2150511.17,20.92
Monday,2,Afternoon,100558,2825086.34,21.79
Tuesday,3,Morning,100236,2580714.13,21.08
Tuesday,3,Evening,129455,3496209.72,25.36
